In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import time
import random 
import pandas as pd
import torch
from torch import nn, cuda, optim
from torchvision import models,transforms,datasets
from torch.utils.data import DataLoader,random_split

In [ ]:
data_dir = './dataset/train'
classes = []
img_per_class = []

for folder in os.listdir(data_dir):    
    classes.append(folder)
    img_per_class.append(len(os.listdir(f'{data_dir}/{folder}')))

num_classes = len(classes)
df = pd.DataFrame({'Classes':classes, 'Examples':img_per_class})
df

In [ ]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
import timm
torch.manual_seed(1557)

_model = timm.create_model("efficientnet_b0", pretrained=True)

model = nn.Sequential(
    _model,
    nn.BatchNorm1d(1000),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1000, num_classes),
)

pytorch_total_params = sum(p.numel() for p in model.parameters())

device = torch.device('cuda:0' if cuda.is_available() else 'cpu')
model.to(device)

print(model)

In [ ]:
train_transform = transforms.Compose([
                                      transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.7, 1.3)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(60),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                      ])

val_transform = transforms.Compose([
                                      transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                    ])

In [ ]:
data = datasets.ImageFolder(data_dir)
train_size = int(len(data)*0.9)
val_size = int((len(data)-train_size))

# Using Sklearn to stratified split in ImageFolder Dataset.
# https://linuxtut.com/en/c6023453e00bfead9e9f/

from sklearn.model_selection import train_test_split

train_indices, val_indices = train_test_split(
    list(range(len(data.targets))), 
    train_size=train_size, # train_size=0.95
    test_size=val_size, # val_size=0.05
    stratify=data.targets,
    random_state=1557,
)
train_data = torch.utils.data.Subset(data, train_indices)
val_data = torch.utils.data.Subset(data, val_indices)

print(f'train size: {len(train_data)}\nval size: {len(val_data)}')

train_data.dataset.transform = train_transform
val_data.dataset.transform = val_transform
batch_size = 64
num_workers = 1
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,num_workers=num_workers)
val_loader = DataLoader(val_data,batch_size=batch_size,shuffle=True,num_workers=num_workers)

In [ ]:
train_distribution_ = np.array(data.targets)[train_indices]
val_distribution_ = np.array(data.targets)[val_indices]

train_distribution = np.zeros(num_classes, dtype=int)
val_distribution = np.zeros(num_classes, dtype=int)

for i in train_distribution_:
  train_distribution[i]+=1

for i in val_distribution_:
  val_distribution[i]+=1

plt.figure(figsize=(25, 8))
for col_idx, bins_ in enumerate([train_distribution, val_distribution, train_distribution + val_distribution]):
  plt.subplot(1, 3, col_idx + 1)
  plt.plot(range(num_classes), bins_)
  plt.title(["train", "val", "total"][col_idx])

plt.show()

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

In [ ]:
from datetime import datetime
now = datetime.now() 
lr_over_time = []

def fit(model,criterion,optimizer, scheduler=None, num_epochs=10):
    global now, lr_over_time
    drive_model_save_path = f'./models/{now.year}_{now.month}_{now.day}_{now.hour}_{now.minute}/'

    print_freq = 250
    start = time.time()
    best_model = model.state_dict()
    best_acc = 0
    train_loss_over_time = []
    val_loss_over_time = []
    train_acc_over_time = []
    val_acc_over_time = []
    
    for epoch in range(num_epochs):
        
        print("\n----- epoch: {}, lr: {} -----".format(epoch, optimizer.param_groups[0]["lr"]))
        batch_time = AverageMeter('Time', ':6.3f')
        acc = AverageMeter('Accuracy', ':.4e')
        progress = ProgressMeter(len(train_loader), batch_time, acc, prefix="Epoch: [{}]".format(epoch))

        for phase in ['train','val']:
            
            if phase == 'train':
                data_loader = train_loader
                model.train()                    # set the model to train mode
                end = time.time()

            else:
                data_loader = val_loader
                model.eval()                    # set the model to evaluate mode
                end = time.time()
            
                
            running_loss = 0.0
            running_corrects = 0.0
            
            # iterate over the data
            for i,(inputs,labels) in enumerate(data_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _,pred = torch.max(outputs,dim=1)
                    loss = criterion(outputs,labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        if scheduler is not None and isinstance(scheduler, torch.optim.lr_scheduler.OneCycleLR): # IF SCHEDULER IS ONE CYCLE
                            lr_over_time.append(scheduler.get_last_lr()[0])
                            if i % print_freq == 0:
                                print(f'Curr batch lr = {scheduler.get_last_lr()[0]}')
                            scheduler.step()
                
                # calculating the loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(pred == labels.data)

                epoch_acc = (running_corrects.double()/len(train_data)).cpu().numpy()
                acc.update(epoch_acc.item(), inputs.size(0))
                
                if phase == 'train':                          
                    batch_time.update(time.time() - end)
                    end = time.time()

                    if i % print_freq == 0:
                        progress.print(i)
                

            if phase == 'train':
                epoch_loss = running_loss/len(train_data)
                train_loss_over_time.append(epoch_loss)
                epoch_acc = (running_corrects.double()/len(train_data)).cpu().numpy()
                train_acc_over_time.append(epoch_acc)


            else:
                epoch_loss = running_loss/len(val_data)
                val_loss_over_time.append(epoch_loss)
                epoch_acc = (running_corrects.double()/len(val_data)).cpu().numpy()
                val_acc_over_time.append(epoch_acc)
          

            print(f'{phase} loss: {epoch_loss:.3f}, acc: {epoch_acc:.3f}')

            if not os.path.exists(drive_model_save_path):
                  os.makedirs(drive_model_save_path)
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                
                torch.save(model, drive_model_save_path + f'model_best.pt')
                # torch.save(model.state_dict(), drive_model_save_path + f'model_best.pt')
            
            torch.save(model, drive_model_save_path + 'model_latest.pt')
            # torch.save(model.state_dict(), drive_model_save_path + 'model_latest.pt')
            

        print('-'*60)
        if scheduler is not None and not isinstance(scheduler, torch.optim.lr_scheduler.OneCycleLR): # IF SCHEDULER IS NOT ONE CYCLE
            print(f"Current epoch #{epoch+1} ended with lr: {scheduler.get_last_lr()[0]}")
            lr_over_time.append(scheduler.get_last_lr()[0])
            scheduler.step()
            
    print('\n') 
    elapsed_time = time.time() - start
    print('==> {:.2f} seconds to train this epoch\n'.format(elapsed_time))
    print(f'best accuracy: {best_acc:.3f}')


    # load best model weights
    model.load_state_dict(best_model)
    loss = {'train':train_loss_over_time, 'val':val_loss_over_time}
    acc = {'train':train_acc_over_time, 'val':val_acc_over_time}

    return model,loss, acc

In [ ]:
epochs = 10

history, loss, acc = fit(model, criterion, optimizer, scheduler=scheduler, num_epochs = epochs)

In [ ]:
# plotting the loss and accuracy curve for each phase
train_loss = loss['train']
val_loss = loss['val']
train_acc = acc['train']
val_acc = acc['val']

epochs_range = range(epochs)
plt.figure(figsize=(20,10))

plt.subplot(1,2,1)
plt.ylim(0,10)
plt.xlim(0,30)
plt.plot(epochs_range, train_loss, label='train_loss')
plt.plot(epochs_range, val_loss, label='val_loss')
plt.legend(loc=0)
plt.title('Loss')

plt.subplot(1,2,2)
plt.plot(epochs_range, train_acc ,label='train_acc')
plt.plot(epochs_range, val_acc, label='val_acc')
plt.legend(loc=0)
plt.ylim(0,1)
plt.xlim(0,30)
plt.title('Accuracy')

In [ ]:
if scheduler is not None:
    if isinstance(scheduler, torch.optim.lr_scheduler.OneCycleLR):
        plt.plot(range(epochs * len(train_loader)), lr_over_time, label="learning rate")
    else:
        plt.plot(epochs_range, lr_over_time, label="learning rate")
    plt.title("lr over time")